In [1]:
## Import the open AI Python dictionaries
!pip install openai==0.28

In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Neha\OneDrive\Desktop\Restauant Recommnder AI\zomato.csv")
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


Approach:
Conversation and Information Gathering: The chatbot will utilize language models to understand and generate natural responses. Through a conversational flow, it will ask relevant questions to gather information about the user's requirements.
Information Extraction: Once the essential information is collected, rule-based functions come into play, extracting top 3 restaurants that best matches the user's needs.
Personalized Recommendation: Leveraging this extracted information, the chatbot engages in further dialogue with the user, efficiently addressing their queries and aiding them in finding the perfect laptop solution.Restaurant

Part 2: System Design

Major functions behind the Chatbot
initialize_conversation(): This initializes the variable conversation with the system message.
get_chat_model_completions(): This takes the ongoing conversation as the input and returns the response by the assistant
moderation_check(): This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
intent_confirmation_layer(): This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
dictionary_present(): This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
compare_laptops_with_user(): This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
initialize_conv_reco(): Initializes the recommendations conversation

Part 3: Implementation

### 3.1 - Import the libraries

Let's start by importing the libraries that we'll require for this project. Following are the ones:
- openai
- pandas
- os, json, ast

The api key is in a text file api_key.txt.

In [8]:
import os,json,ast
import openai
import pandas as pd

In [9]:
# read the API key
openai.api_key = open(r"C:\Users\Neha\OneDrive\Desktop\Shop Assist AI\OPENAI_API_Key.txt").read().strip()

3.2 - Implementing Intent Clarity and Intent Confirmation Layers

Let's start with the first part of the implementation - building the intent clarity and intent confirmation layers. As mentioned earlier, this layer helps in identifying the user requirements and passing it on to the product matching layer.

initialize_conversation(): This initializes the variable conversation with the system message. Using prompt engineering and chain of thought reasoning, the function will enable the chatbot to keep asking questions until the user requirements have been captured in a dictionary. It also includes Few Shot Prompting(sample conversation between the user and assistant) to align the model about user and assistant responses at each step.


In [11]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    system_message = """
    You are a chatbot designed to gather user requirements for a restaurant recommendation project. 
    Use chain-of-thought reasoning to ask step-by-step questions and ensure that all user requirements 
    are captured in the following dictionary format:
    
    {
        'Location': <string>, 
        'cuisines': <string>, 
        'dish_liked': <string>, 
        'approx_cost': <numerical>
    }
    
    #### Instructions:
    1. All keys except 'approx_cost' should be string values provided by the user.
    2. 'approx_cost' must be a numerical value ≥ 200 INR. If the user provides a value below 200, explain that no restaurants are available in that range.
    3. Do not randomly assign values. Infer them directly from the user's responses.
    4. Validate inputs for all keys and ask clarifying questions if needed.

    #### Chain of Thought Process:
    - **Step 1**: Start by welcoming the user and ask questions to gather information for one or more keys (e.g., location, cuisine). 
    - **Step 2**: Identify missing keys and ask follow-up questions to fill them.
    - **Step 3**: Confirm that all key-value pairs are complete and valid. If unsure about any value, ask for clarification.
    - **Step 4**: Output the final dictionary containing all user requirements.

    #### Example Conversation:
    User: "I want to eat Chinese food."
    Assistant: "Great! Let’s start. Could you tell me your location?"
    User: "Delhi"
    Assistant: "Thanks! What’s your favorite dish in Chinese cuisine?"
    User: "Noodles"
    Assistant: "Noted. Lastly, what’s your budget for this meal?"
    User: "1000"
    Assistant: "Here’s what I’ve gathered: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'Noodles', 'approx_cost': 1000}. Is this correct?"
    """

    # Initialize conversation with system message
    return [{"role": "system", "content": system_message}]


Let's see what does initialize_conversation() does. We have added a prefix debug_ to each of the variables so that we can play around with the inputs and outputs and it doesn't disturb the main function.

In [13]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are a chatbot designed to gather user requirements for a restaurant recommendation project. \n    Use chain-of-thought reasoning to ask step-by-step questions and ensure that all user requirements \n    are captured in the following dictionary format:\n    \n    {\n        \'Location\': <string>, \n        \'cuisines\': <string>, \n        \'dish_liked\': <string>, \n        \'approx_cost\': <numerical>\n    }\n    \n    #### Instructions:\n    1. All keys except \'approx_cost\' should be string values provided by the user.\n    2. \'approx_cost\' must be a numerical value ≥ 200 INR. If the user provides a value below 200, explain that no restaurants are available in that range.\n    3. Do not randomly assign values. Infer them directly from the user\'s responses.\n    4. Validate inputs for all keys and ask clarifying questions if needed.\n\n    #### Chain of Thought Process:\n    - **Step 1**: Start by welcoming the user and ask questions to 

get_chat_model_completions(): This takes the ongoing conversation as the input and returns the response by the assistant

In [15]:
def get_chat_model_completions(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,  # This is the degree of randomness of the model's output
        max_tokens=3000
    )
    return response.choices[0].message.content  # Access the `content` attribute

In [16]:
debug_introduction = get_chat_model_completions(debug_conversation)
print(debug_introduction)


Hello! I'm here to help you find the perfect restaurant for you. Let's get started! 

Could you please tell me your preferred location for dining out?


In [17]:
debug_user_input = "Hi, I am Neha. I want to eat pasta."

In [18]:
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
print(debug_response_assistant)

Great, Neha! Let's get started. 
Could you please tell me your location?


moderation_check(): This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, you can add a break statement to end the conversation.

In [20]:
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_output = response["results"][0]
    if moderation_output["flagged"] == True:
        return "Flagged"
    else:
        return "Not Flagged"

In [21]:
debug_moderation = moderation_check(debug_user_input)
print(debug_moderation)

Not Flagged


In [22]:
print(moderation_check("I want to kill Ravan."))
print(moderation_check("I can do anything for a hot chocolate right now."))

Flagged
Not Flagged


In [23]:
moderation_check(debug_response_assistant)

'Not Flagged'

As mentioned earlier, you need to understand the user's profile, which essentially means that all the features: GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget are captured or not. Let's look at the function that helps us verify that.

intent_confirmation_layer(): This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not
GPU intensity
Display quality
Portability
Multitasking
Processing speed
Budget

In [25]:
def intent_confirmation_layer(response_assistant):
    """
    Evaluates if the provided input contains all the required keys and whether their values are correctly filled.
    """
    prompt = f"""
    You are a senior evaluator with an eye for detail.
    Evaluate if the provided input contains all the required keys and whether their values are correctly filled:
    
    Required keys and conditions:
    - 'Location': should be a valid city name (e.g., 'Delhi', 'Mumbai').
    - 'cuisines': should describe a type of cuisine (e.g., 'Italian', 'Chinese').
    - 'dish_liked': should describe a dish name (e.g., 'Pasta', 'Biryani').
    - 'approx_cost': should be a numerical value greater than or equal to 200.
    
    Input: {response_assistant}
    
    Output the result as 'Yes' if all keys are present and have correctly filled values; otherwise, output 'No'.
    Examples:
    - Input: {{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'Noodles', 'approx_cost': 1000}}
      Output: Yes
    - Input: {{'Location': '', 'cuisines': 'Chinese', 'dish_liked': '', 'approx_cost': 100}}
      Output: No
    Only output a one-word string: Yes or No.
    """

    confirmation = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a senior evaluator with an eye for detail."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return confirmation["choices"][0]["message"]["content"].strip()


In [26]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
print(debug_confirmation)

Input: Great, Neha! Let's get started. Could you please tell me your location?

Output: No


In [27]:
response_assistant_1 = {
    "Location": "Delhi",
    "cuisines": "Italian",
    "dish_liked": 'Pasta',
    "approx_cost": "1000"
}

print("Input:", response_assistant_1)
print("Output:", intent_confirmation_layer(response_assistant_1))


Input: {'Location': 'Delhi', 'cuisines': 'Italian', 'dish_liked': 'Pasta', 'approx_cost': '1000'}
Output: No


Let's now look at the working of dictionary_present().

dictionary_present(): This function checks if the final understanding of user's profile is returned by the chatbot is a Python dictionary or not. This is important as it'll be used later on for finding the right laptops using dictionary matching.

In [29]:
def dictionary_present(response):
    delimiter = "####"
    example_user_req = {
        'Location': 'Delhi', 
        'cuisines': 'Chinese', 
        'dish_liked': 'Pasta', 
        'approx_cost': 800
    }
    
    prompt = f"""
    You are a Python expert. You are provided with an input.
    Your task is to check if there is a Python dictionary present in the string.
    The dictionary should have the following format: {example_user_req}.
    Extract and return only the Python dictionary from the input.
    The output must match the format: {example_user_req}.
    Ensure the output contains the exact keys and values present in the input.

    Here are some sample input-output pairs for better understanding:
    {delimiter}
    Input: Location: Mumbai, cuisines: Italian, dish_liked: Pizza, approx_cost: 1200
    Output: {{'Location': 'Mumbai', 'cuisines': 'Italian', 'dish_liked': 'Pizza', 'approx_cost': 1200}}

    Input: {{'Location': 'Bangalore', 'cuisines': 'South Indian', 'dish_liked': 'Dosa', 'approx_cost': 500}}
    Output: {{'Location': 'Bangalore', 'cuisines': 'South Indian', 'dish_liked': 'Dosa', 'approx_cost': 500}}

    Input: Here is your user profile: 'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'Noodles', 'approx_cost': 1000
    Output: {{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'Noodles', 'approx_cost': 1000}}
    {delimiter}

    Here is the input: {response}
    """

    # Use ChatCompletion for GPT-3.5-Turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a Python expert."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"].strip()


In [30]:
debug_response_assistant_n = {"Location": "Delhi","cuisines": "Chinese","dish_liked": "pasta","approx_cost": "800"}
response_dict_n = dictionary_present(debug_response_assistant_n)
print("Extracted Dictionary:", response_dict_n)

Extracted Dictionary: Here is the Python code to extract and return the Python dictionary from the given input:

```python
import ast

input_str = "{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}"

try:
    output_dict = ast.literal_eval(input_str)
    if isinstance(output_dict, dict) and all(key in output_dict for key in ['Location', 'cuisines', 'dish_liked', 'approx_cost']):
        print(output_dict)
    else:
        print("No valid Python dictionary found in the input.")
except ValueError:
    print("No valid Python dictionary found in the input.")
```

When you run this code with the provided input string, it will extract and return the Python dictionary:

Output:
```python
{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}
```


In [31]:
response_dict_n = dictionary_present(response_assistant_1)
print(response_dict_n)

```python
def extract_dict_from_input(input_str):
    start_idx = input_str.find('{')
    end_idx = input_str.rfind('}') + 1
    dictionary_str = input_str[start_idx:end_idx]
    
    try:
        extracted_dict = eval(dictionary_str)
        return extracted_dict
    except SyntaxError:
        return None

input_str = "{'Location': 'Delhi', 'cuisines': 'Italian', 'dish_liked': 'Pasta', 'approx_cost': '1000'}"
output_dict = extract_dict_from_input(input_str)
print(output_dict)
``` 

Output:
{'Location': 'Delhi', 'cuisines': 'Italian', 'dish_liked': 'Pasta', 'approx_cost': '1000'}
```


In [32]:
debug_conversation = initialize_conversation()
debug_introduction = get_chat_model_completions(debug_conversation)
debug_user_input = "Hi, I am Neha. I want pasta."
debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "content": debug_user_input})
debug_response_assistant = get_chat_model_completions(debug_conversation)
debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "assistant", "content": debug_response_assistant})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
response_dict_n = dictionary_present(debug_response_assistant_n)

3.3 Implementing the Product Mapping and Information Extraction Layers
In this section, we take in the output of the previous layers, i.e. the user requirements, which is in the format of a Python dictionary, and extract the top 3 Restaurant recommendations based on that. Here are the functions that we will use to help us implement the information extraction and product matching layers

product_map_layer(): This function is responsible for extracting key features and criteria from the data.

In [34]:
def restaurant_map_layer(restaurant_description):
    delimiter = "#####"
    restaurant_parameters = {
        "cuisines": "(Type of cuisines offered, e.g., Italian, Chinese, Indian)",
        "approx_cost": "(Average cost for two people in INR)",
        "dish_liked": "(Popular dishes, e.g., Pizza, Biryani)",
        "location": "(City or locality of the restaurant)",
        "rating": "(Overall rating, e.g., 4.1/5)",
    }

    cost_classification = {
        "low": "less than 500 INR",
        "medium": "between 500 INR and 1500 INR",
        "high": "greater than 1500 INR"
    }

    prompt = f"""
    You are a Restaurant Classifier whose job is to extract key features of a restaurant based on its description and classify them.

    To analyze a restaurant, perform the following steps:
    Step 1: Extract the restaurant's primary features from the description: {restaurant_description}.
    Step 2: Store the extracted features in this structure: {restaurant_parameters}.
    Step 3: Classify the 'approx_cost' into one of these categories: {cost_classification}.
    Step 4: Return the final structured classification as a Python dictionary.

    Example Input:
    {delimiter}
    Input: "Jalsa is a highly rated restaurant located in Banashankari, Bangalore. Known for its North Indian and Mughlai cuisines, it offers a cozy dining experience. Popular dishes include Pasta and Lunch Buffet. The average cost for two people is around 800 INR, and the restaurant boasts a rating of 4.1/5. It is a great option for casual dining."
    Output: {{
        'cuisines': 'North Indian, Mughlai',
        'approx_cost': 'medium',
        'dish_liked': 'Pasta, Lunch Buffet',
        'location': 'Banashankari, Bangalore',
        'rating': '4.1/5'
    }}
    {delimiter}

    Now analyze the following restaurant description and classify its features:
    {restaurant_description}
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=1000
    )
    return response["choices"][0]["message"]["content"].strip()


In [35]:
import pandas as pd

# Load the restaurant data
restaurant_df = pd.read_csv(r"C:\Users\Neha\OneDrive\Desktop\Restauant Recommnder AI\zomato.csv")

# Define a function to map structured data into a dictionary
def map_restaurant_features(row):
    """
    Maps the structured data of a restaurant into a dictionary of features.
    """
    return {
        'cuisines': row['cuisines'] if pd.notna(row['cuisines']) else 'Unknown',
        'approx_cost': row['approx_cost(for two people)'] if pd.notna(row['approx_cost(for two people)']) else 'Unknown',
        'dish_liked': row['dish_liked'] if pd.notna(row['dish_liked']) else 'Unknown',
        'location': row['location'] if pd.notna(row['location']) else 'Unknown',
        'rating': row['rate'] if pd.notna(row['rate']) else 'Unknown'
    }

# Apply the function to each row to create the 'restaurant_feature' column
restaurant_df['restaurant_feature'] = restaurant_df.apply(map_restaurant_features, axis=1)

# Save the updated DataFrame to a new CSV file
restaurant_df.to_csv("updated_restaurant.csv", index=False, header=True)

print("Updated DataFrame saved to 'updated_restaurant.csv'")


Updated DataFrame saved to 'updated_restaurant.csv'


In [36]:
import pandas as pd

# Load the restaurant data (updated_restaurant.csv) into a DataFrame
restaurant_df = pd.read_csv(r"C:\Users\Neha\updated_restaurant.csv")

# Rename the column from 'approx_cost(for two people)' to 'approx_cost_for_two_people'
restaurant_df.rename(columns={'approx_cost(for two people)': 'approx_cost_for_two_people'}, inplace=True)

# Print the updated columns to verify
print(restaurant_df.columns)


Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost_for_two_people', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)', 'restaurant_feature'],
      dtype='object')


In [37]:
import ast
import re

def extract_dictionary_from_string(string):
    """
    Extracts a dictionary with specific keys ('Location', 'cuisines', 'dish_liked', 'approx_cost') from a string.
    """
    # If the string doesn't contain curly braces, wrap it in braces
    if not string.startswith("{"):
        string = "{" + string + "}"
    
    print(f"String after wrapping with braces: {string}")  # Debugging statement
    
    # Define the regex pattern to match a dictionary structure
    regex_pattern = r"\{[^{}]+\}"

    # Find all the dictionary-like patterns in the string
    dictionary_matches = re.findall(regex_pattern, string)
    print(f"Matches found: {dictionary_matches}")  # Debugging statement

    # If a match is found, process the first match
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        
        # Clean the string by removing any extra spaces, newlines, tabs, etc.
        dictionary_string = dictionary_string.strip()

        print(f"Extracted dictionary string: {dictionary_string}")  # Debugging statement
        
        try:
            # Convert the string to a dictionary using ast.literal_eval()
            extracted_dict = ast.literal_eval(dictionary_string)

            # Process the extracted dictionary to match the expected format
            if 'approx_cost' in extracted_dict:
                # Convert 'approx_cost' to a numeric value
                extracted_dict['approx_cost'] = float(extracted_dict['approx_cost'])
            
            # Normalize the dictionary keys to match the required structure
            final_dict = {
                'Location': extracted_dict.get('location', '').capitalize(),
                'cuisines': extracted_dict.get('cuisines', '').capitalize(),
                'dish_liked': extracted_dict.get('dish_liked', '').capitalize(),
                'approx_cost': extracted_dict.get('approx_cost', 0)  # Ensure it's numeric
            }
            
            return final_dict
        except Exception as e:
            print(f"Error while parsing dictionary: {e}")
            return None
    return None


# Example string
string = " \n \t           Output: {{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}ad"

# Extract the dictionary
extracted_dict = extract_dictionary_from_string(string)
print(extracted_dict)

String after wrapping with braces: { 
 	           Output: {{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}ad}
Matches found: ["{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}"]
Extracted dictionary string: {'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}
{'Location': '', 'cuisines': 'Chinese', 'dish_liked': 'Pasta', 'approx_cost': 800.0}


In [38]:
string = " \n \t           Output: {{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}ad"

In [39]:
example_2 = extract_dictionary_from_string(string)
print(example_2)

String after wrapping with braces: { 
 	           Output: {{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}ad}
Matches found: ["{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}"]
Extracted dictionary string: {'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}
{'Location': '', 'cuisines': 'Chinese', 'dish_liked': 'Pasta', 'approx_cost': 800.0}


In [40]:
string_1 = "Location: 'Delhi', cuisines: 'Chinese', dish_liked: 'pasta', approx_cost: '1000'"
extracted_dict = extract_dictionary_from_string(string)
print(f"Extracted dictionary: {extracted_dict}")

String after wrapping with braces: { 
 	           Output: {{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}ad}
Matches found: ["{'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}"]
Extracted dictionary string: {'Location': 'delhi', 'cuisines': 'chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}
Extracted dictionary: {'Location': '', 'cuisines': 'Chinese', 'dish_liked': 'Pasta', 'approx_cost': 800.0}


In [41]:
def compare_restaurants_with_user(user_req_string):
    # Load the restaurant data (updated_restaurant.csv) into a DataFrame
    restaurant_df = pd.read_csv(r"C:\Users\Neha\updated_restaurant.csv")

    # Extract the user requirements from the input string
    user_requirements = extract_dictionary_from_string(user_req_string)
    
    if user_requirements is None:
        print("Error: User requirements could not be extracted. Please check the input string format.")
        return None  # Return None or an empty response to indicate failure
    
    print(f"User Requirements: {user_requirements}")

    # Extract the user's 'approx_cost' (which is the budget)
    approx_cost = float(user_requirements.get('approx_cost', 0))  # Ensure it's a number
    print(f"Approx Cost (Budget): {approx_cost}")
    
    # Create a copy of the restaurant DataFrame for filtering
    filtered_restaurants = restaurant_df.copy()
    
    # Normalize the column names
    approx_cost_column = 'approx_cost_for_two_people'  # The normalized name
    if approx_cost_column in filtered_restaurants.columns:
        # Clean up the 'approx_cost_for_two_people' column and convert to float
        filtered_restaurants[approx_cost_column] = filtered_restaurants[approx_cost_column].str.replace(',', '').astype(float)
    else:
        print("Error: 'approx_cost_for_two_people' column is missing.")
        return None  # Handle this case as needed
    
    # Filter restaurants by budget (approx_cost)
    filtered_restaurants = filtered_restaurants[filtered_restaurants[approx_cost_column] <= approx_cost].copy()

    # Mappings for comparison (this assumes 'low', 'medium', 'high' values are in the data)
    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }

    # Create a 'Score' column initialized to 0
    filtered_restaurants['Score'] = 0

    # Loop through each restaurant in the filtered DataFrame
    for index, row in filtered_restaurants.iterrows():
        # Get the features of the restaurant
        restaurant_features = extract_dictionary_from_string(row['restaurant_feature'])
        
        # Initialize the score
        score = 0

        # Compare user requirements with restaurant features
        for key, user_value in user_requirements.items():
            if key.lower() == 'approx_cost':
                continue  # Skip comparison for 'approx_cost' as it's handled separately
            
            # Get the restaurant value for the key
            restaurant_value = restaurant_features.get(key, None)
            
            if restaurant_value:
                # Map the values to numerical scores for comparison
                restaurant_mapping = mappings.get(restaurant_value.lower(), -1)
                user_mapping = mappings.get(user_value.lower(), -1)

                # Compare the restaurant value to the user's value
                if restaurant_mapping >= user_mapping:
                    score += 1

        # Assign the computed score to the restaurant's row
        filtered_restaurants.loc[index, 'Score'] = score

    # Sort the filtered restaurants by their score in descending order
    top_restaurants = filtered_restaurants.drop('restaurant_feature', axis=1)
    top_restaurants = top_restaurants.sort_values('Score', ascending=False).head(3)

    # Return the top restaurants as JSON
    return top_restaurants.to_json(orient='records')


In [42]:
user_req_string = "{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}"
comparision_result = compare_restaurants_with_user(user_req_string)
print (user_req_string)

String after wrapping with braces: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}
Matches found: ["{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}"]
Extracted dictionary string: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}
User Requirements: {'Location': '', 'cuisines': 'Chinese', 'dish_liked': 'Pasta', 'approx_cost': 1000.0}
Approx Cost (Budget): 1000.0
Error: 'approx_cost_for_two_people' column is missing.
{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}


In [43]:
top_3_restaurants = compare_restaurants_with_user(response_dict_n)
print(top_3_restaurants)

String after wrapping with braces: {def extract_dictionary(input_str):
    import ast

    # Try to extract Python dictionary from the input string
    try:
        extracted_dict = ast.literal_eval(input_str)
        
        # Check if the extracted object is a dictionary
        if isinstance(extracted_dict, dict):
            # Check if the dictionary has the required keys
            if all(key in extracted_dict for key in ['Location', 'cuisines', 'dish_liked', 'approx_cost']):
                return extracted_dict
    except (SyntaxError, ValueError):
        pass
    
    return None

input_str = "{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}"
output_dict = extract_dictionary(input_str)

print(output_dict)   # Output: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}}
Matches found: ["{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '800'}", "{'Location': 'Delhi'

In [44]:
def verify_recommendations(user_req_string, top_restaurants_json):
    """
    Verifies if the recommended restaurants meet the user's requirements.
    
    Parameters:
    user_req_string (str): The input string containing the user's preferences.
    top_restaurants_json (str): JSON string containing the top recommended restaurants.
    
    Returns:
    str: Verification result message.
    """
    import json
    
    # Parse the top restaurants JSON into a DataFrame or dictionary
    top_restaurants = pd.read_json(top_restaurants_json)

    # Extract the user requirements from the input string
    user_requirements = extract_dictionary_from_string(user_req_string)

    if user_requirements is None:
        return "Error: User requirements could not be extracted. Please check the input string format."

    # Check if top restaurants list is empty
    if top_restaurants.empty:
        return "No restaurants found that meet the requirements."

    # Initialize the verification message
    verification_message = "Verification Result:\n"
    all_requirements_met = True  # Flag to check if all requirements are met

    # Check each restaurant's attributes against user requirements
    for index, row in top_restaurants.iterrows():
        restaurant_name = row['name']
    


In [45]:
# Assuming top_3_restaurants is the output of compare_restaurants_with_user
user_req_string = "{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}"

# Get the recommendations as JSON (using the compare_restaurants_with_user function)
top_restaurants_json = compare_restaurants_with_user(user_req_string)

# Now validate the recommendations
validated_data = verify_recommendations(user_req_string, top_restaurants_json)
print(validated_data)


String after wrapping with braces: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}
Matches found: ["{'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}"]
Extracted dictionary string: {'Location': 'Delhi', 'cuisines': 'Chinese', 'dish_liked': 'pasta', 'approx_cost': '1000'}
User Requirements: {'Location': '', 'cuisines': 'Chinese', 'dish_liked': 'Pasta', 'approx_cost': 1000.0}
Approx Cost (Budget): 1000.0
Error: 'approx_cost_for_two_people' column is missing.


ValueError: Invalid file path or buffer object type: <class 'NoneType'>